# Identify parcel runs start locations

In [16]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

%matplotlib inline

#### Parameters

In [4]:
# domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179
isize = imax - imin
jsize = jmax - jmin

# colors:
land_color = '#a9a7a2'

#### Files

In [7]:
# ANHA12 mesh grid:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20220317.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
mesh_bathy   = mesh['tmask'][0,:,:,:].values
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = mesh['nav_lev'].values

#### Functions

In [10]:
def save_file(lon, lat, depth, filepath):    
    file_write = xr.Dataset(
        {'longitude' : (("x"), lon),
         'latitude'  : (("y"), lat),
         'depth'     : (("z"), depth)}, 
        coords = {"z": depths,
                  "y": lat,
                  "x": lon},
    )
    file_write.pint.quantify({"longitude": "degrees East", 
                              "latitude" : "degrees North",
                              "depth"    : "m"})
    file_write.pint.dequantify()
    file_write.to_netcdf(f'{filepath}')
    
    return

In [31]:
def plot_locations(release_lon, release_lat, depth=0):

    fig, ax1, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/pb-surface.pickle','rb'))

    x_plot, y_plot     = proj1(release_lon, release_lat)
    x_ANHA12, y_ANHA12 = proj1(mesh_lon, mesh_lat)

    ax1.plot(x_ANHA12[imin:imax,jmin], y_ANHA12[imin:imax,jmin], '-k', linewidth=1.0)
    ax1.plot(x_ANHA12[imin:imax,jmax], y_ANHA12[imin:imax,jmax], '-k', linewidth=1.0)
    ax1.plot(x_ANHA12[imin,jmin:jmax], y_ANHA12[imin,jmin:jmax], '-k', linewidth=1.0)
    ax1.plot(x_ANHA12[imax,jmin:jmax], y_ANHA12[imax,jmin:jmax], '-k', linewidth=1.0)

    proj1.contourf(x_ANHA12, y_ANHA12, bathy_masked[depth,:,:], colors=[land_color], zorder=1) #Apply land mask

    ax1.scatter(x_plot, y_plot, zorder=4, s=5)
    
    return

#### Main work

In [17]:
release_lon   = [-130, -145, -70, -55] # east
release_lat   = [75, 74, 75, 65]  # north

#### Save file

In [ ]:
# Write release longitudes, latitudes, and depths to file:
save_file(lon, lat, depth, '/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/.nc')